In [76]:
import sys
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import model_selection, metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm

In [84]:
torch.cuda.is_available()

True

In [2]:
import urllib
from sklearn.datasets import fetch_mldata

try:
    mnist = fetch_mldata('MNIST original')
except urllib.error.HTTPError as ex:
    from six.moves import urllib
    from scipy.io import loadmat
    import os
    print('Could not download MNIST data from mldata.org, trying alternative...')
    mnist_alternative_url = 'https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat'
    mnist_path = './mnist-original.mat'
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, 'wb') as f:
        content = response.read()
        f.write(content)
    mnist_raw = loadmat(mnist_path)
    mnist = {
        "data": mnist_raw['data'].T,
        "target": mnist_raw['label'][0],
        "COL_NAMES": ['label', 'data'],
        "DESCR": 'mldata.org dataset: mnist-original',
    }
    print('Success!')

In [85]:
mnist['data'] = mnist['data'].astype(np.float32).reshape(len(mnist['data']), 1, 28, 28) # image data
mnist['data'] /= 255
mnist['target'] = mnist['target'].astype(np.int32) # label data
mnist['data'].shape, mnist['target'].shape

((70000, 1, 28, 28), (70000,))

In [86]:
# train data size : validation data size= 8 : 2
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(mnist['data'], mnist['target'], test_size=0.2)

# DataLoader化
batch_size = 1000
train = torch.utils.data.TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid = torch.utils.data.TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size)
len(train_loader), len(valid_loader)

(56, 14)

In [87]:
# モデルクラス定義

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        # 画像を畳み込みを行うまで
        self.head = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 5), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(5, 5), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        # 畳み込みで得られたベクトルを出力層に順伝播させるまで
        self.tail = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 10)
        )
    
    def __call__(self, x):
        
        h = self.head(x)
        h = h.view(-1, 320)
        h = self.tail(h)
        y = F.log_softmax(h)
        return y

In [88]:
# test

model = CNN()
xs, ys = iter(train_loader).next()
print(xs.size())
preds = model(Variable(xs))
preds.shape

torch.Size([1000, 1, 28, 28])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1000, 10])

In [89]:
gpu = -1

model = CNN()

if gpu >= 0:
    model.cuda(gpu)
    
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [90]:
def train(train_loader):
    
    #model.train()
    running_loss = 0
    
    for batch_idx, (xs, ys) in enumerate(train_loader):
        
        if gpu >= 0:
            xs = Variable(xs.cuda(gpu))
            ys = Variable(ys.cuda(gpu)).long()
        else:
            xs = Variable(xs)
            ys = Variable(ys).long()

        optimizer.zero_grad()
        preds = model(xs)

        loss = criterion(preds, ys)
        running_loss += loss.data[0]

        loss.backward()
        optimizer.step()

    train_loss = running_loss / len(train_loader)
    
    return train_loss

def valid(valid_loader):
    
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (xs, ys) in enumerate(valid_loader):
        
        if gpu>=0:
            xs = Variable(xs.cuda(gpu), volatile=True)
            ys = Variable(ys.cuda(gpu), volatile=True).long()
        else:
            xs = Variable(xs, volatile=True)
            ys = Variable(ys, volatile=True).long()

        preds = model(xs)

        loss = criterion(preds, ys)
        running_loss += loss.data[0]

        _, pred_labels = torch.max(preds.data, 1)
        correct += (pred_labels == ys.data).sum()
        total += ys.size(0)

    val_loss = running_loss / len(valid_loader)
    val_acc = correct / total
    
    return val_loss, val_acc

In [91]:
# train

epoch_num = 5

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    loss = train(train_loader)
    val_loss, val_acc = valid(valid_loader)
    
    if (epoch+1) % 1 == 0:
        tqdm.write('epoch:\t{}\tloss:\t{}\tval loss:\t{}\tval acc:\t{}'.format(epoch+1, loss, val_loss, val_acc))

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch:	1	loss:	2.3022892475128174	val loss:	2.3016395568847656	val acc:	0
epoch:	2	loss:	2.3013088703155518	val loss:	2.301541566848755	val acc:	0
epoch:	3	loss:	2.3012452125549316	val loss:	2.3016016483306885	val acc:	0
epoch:	4	loss:	2.3012397289276123	val loss:	2.301548480987549	val acc:	0
epoch:	5	loss:	2.30120587348938	val loss:	2.301722764968872	val acc:	0
100%|██████████| 5/5 [00:38<00:00,  7.63s/it]


In [83]:
# train

epoch_num = 5

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    # mini-batch
    total_loss = 0
    for i, data in enumerate(train_loader):
        
        x, y = data
        x, y = Variable(x), Variable(y).long()
        
        if gpu >= 0:
            x.cuda(gpu)
            y.cuda(gpu)
            
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        total_loss += loss.data[0]
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 1 == 0:
        tqdm.write('epoch:\t{}\ttotal loss:\t{}'.format(epoch+1, total_loss))

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch:	1	total loss:	128.9339599609375
epoch:	2	total loss:	128.87838745117188      
epoch:	3	total loss:	128.8729248046875       
epoch:	4	total loss:	128.8755645751953       
epoch:	5	total loss:	128.87063598632812      
100%|██████████| 5/5 [00:35<00:00,  7.02s/it]


In [ ]:
# 予測

print("\nPredict")
def predict(model, x):
    x_ = np.array([x], dtype="float32")
    x_ = torch.from_numpy(x_)
    x_ = Variable(x_)
    y = model(x_)
    _, y = torch.max(y.data, 1)
    plt.figure(figsize=(1, 1))
    plt.imshow(x[0], cmap=cm.gray_r)
    plt.show()
    print("y:\t{}\n".format(y[0]))

idx = np.random.choice((70000-N), 10)
for i in idx:
    predict(model, test_x[i])